In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
coordinates

[(-42.729686707563026, 106.84572481654413),
 (-39.37259483660639, -46.475331421154266),
 (42.696183565922524, -53.31156172847386),
 (-16.721491630448085, -44.231491815462675),
 (-75.02362667247394, 70.23983044774133),
 (80.75804158743088, -45.10618730961542),
 (-79.16262638793614, -177.63188329794738),
 (-22.52392916534302, 167.51254079676232),
 (-26.503894203413942, 100.58851548273083),
 (-5.225347276821424, 152.7332356221143),
 (-64.85860080102977, -67.85795254324965),
 (31.971783813702004, -116.64226093050324),
 (79.71743150960458, -114.04210095013212),
 (-47.39411023563817, -154.7268673162659),
 (-38.18240363806409, 58.83960630962747),
 (-9.72154773126502, 170.86259960071783),
 (-19.47133172810838, 4.4307179145271505),
 (30.28052184164528, 55.7637549488465),
 (-24.14461370293668, -169.9535468451843),
 (47.69160972787034, 33.531072690142736),
 (35.95834023780594, -17.479237430671247),
 (-60.7219753888059, -153.94218014945267),
 (-32.49156852534347, -20.30479405056755),
 (-84.4012561

In [6]:
from citipy import citipy

In [7]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

600